## 라이브러리 읽기

In [1]:
import pandas as pd
import numpy as np


from konlpy.tag import Hannanum # 토크나이징
import gensim # 토픽 모델링
import re # 정규표현식

import gensim.corpora as corpora

from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel

from pprint import pprint

import pyLDAvis
from pyLDAvis import gensim

In [2]:
hannanum = Hannanum()

#### 데이터 불러오기

In [3]:
df = pd.read_csv("data/battlenetData.csv", encoding='utf-8',header=None)

In [4]:
df.columns = ['link_id', 'user_id', 'timestamp', 'title', 'contents']

In [5]:
df.tail(20)

,link_id,user_id,timestamp,title,contents
5680,4564482455,DTworld,2017.04.24 14시 04분,골렘종류의 캐릭터가 나왔으면 좋겠어요,[산악거인 같은거 나왔으면 좋겠어요]
5681,4564482452,메오후,2017.04.24 14시 02분,블코 정신 못차렸나요?,"[홍보방송을 아프리카에서 하고, 그것도 새벽에 한다고요?? 남들 다 자는 시간에다가..."
5682,4564482451,zujak,2017.04.24 14시 02분,롤 bj한테 왜 시키냐?,[겐지 나왔으면 히오스랑 관련 없는 bj시킬거면 차라리 김혜성이나 오버워치 bj라도...
5683,4564514332,Cosmos,2017.04.24 11시 03분,루나라 좀 상향해주세요,[한타만 벌어지면 체력 제일 약한 암노루부터 죽이고 시작하네 ㅠㅠ노루고기 살살 녹는...
5684,4564514290,디모엘리스는사랑,2017.04.24 09시 05분,히오스 2.0 구체적인 날짜,[히오스 홈페이지에 4/25일에 2.0패치 된다고 올라왔는데 이 날짜는 한국표준시인...
5685,4564494456,셜록,2017.04.23 14시 52분,2.0에서 구매한 스킨이나 영웅,[2.0테스트 서버에서 구매한 영웅들이나 스킨이 일반 서버로 전환될때 넘어오나요아니...
5686,4564514177,ZASK,2017.04.23 14시 22분,2.0업데이트 이후 궁금한점이있습니다.,[현재 히어로즈오브더스톰에서 계정레벨 12렙을 달성하면 하스스톤에서 사용가능한 카드...
5687,4564387780,Shrimp,2017.02.07 20시 23분,빠른시전 단축키에 있어서,[히오스에서 빠른 시전 설정을 하면 사거리를 보기 힘들어서 마우스 커서로 확인 해야...
5688,4564504099,소스바,2017.04.23 01시 22분,히오스 베타 관련..,[안녕하세요 히오스 베타 2.0을 즐기는 유저입니다혹시 베타 테스트 이벤트중에서 ...
5689,4564503916,드립슈,2017.04.22 21시 01분,랭크 아이디 가리게 해주셈,[롤처럼 소환사1 이렇게]


## 데이터 전처리 하기

In [8]:
for i in range(len(df["contents"])):
    df['contents'][i] = re.sub("<.+>", "", df['contents'][i])
    df['contents'][i] = re.sub("[[\]]", "",df['contents'][i])

/Users/Yeoni/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Yeoni/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [23]:
df.drop(df.index[1017])

,link_id,user_id,timestamp,title,contents
0,4564557252,비전력이부조카당,2017.05.06 05시 21분,"아바투르 유저인데, 하나 건의합니다.","아바투르 유저인데, 하나 건의합니다.새로 생긴 하나무라 맵...뭐 이것도 맵 자체에..."
1,4567016696,네이팜데쓰,2018.09.08 16시 53분,"매칭 시스템, 아무리 생각해도 문제가 많습니다",매칭좀 제대로 해주세요.9연패가 말이 되는 처사입니까 며칠전에 16연승했다고 벌을 ...
2,4567016695,souse,2018.09.08 16시 22분,ㅈ같은 매칭좀 어캐좀 해봐요,왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야하죠? 왜 심지어 못하는새키랑 ...
3,4567026707,Best,2018.09.08 15시 13분,아나 스킨관련..,신 스킨 살무사 아나 수면총 색 고정말고. 스킨 색 대로 해주셧으면 좋겠습니다. ...
4,4567036673,세월,2018.09.07 13시 58분,5인팟 + 승률50%집착매칭 + 밸런스 파괴 = 빠대무간지옥,승률 50% 만든다고 조합 !@내놓고5인팟은 여전히 솔팟 사이에서 개판치고다니고너프...
5,4567026700,Ditto,2018.09.08 09시 06분,영웅리그 매칭에서 대만인들,대만인 새끼들좀 매칭 안되게 해라.말도 안통하는 머저리 새끼들하고 왜 해야하는건데빠...
6,4567046681,JELAWAT,2018.09.08 07시 09분,신영웅 그만만들고 기존영웅 손좀보시죠^^,신영웅 이제 그만좀만드시고요. 기존영웅 쓸수나있게 손좀보시죠^^ 뭐가 더 급한건지 ...
7,4567006799,요우몽,2018.09.06 16시 47분,망한맵들은 리메이크가 답이 아니다,삭제가 답이지 죽광도 리메이크해도 망맵 하나무라도 리메이크해도 망맵블랙하트는 예전에...
8,4567036686,파워디지몬,2018.09.08 04시 26분,히오스 영웅리그 개선안,HOTS는 이전부터 영웅리그에 대한 문제가 많았습니다.크게 정리하자면 4가지가 있습...
9,4567036678,인생이지겨워,2018.09.07 23시 59분,다른 영웅 선택되는 버그,분명 픽창에서는 제가 하고싶은 영웅을 선택했는데 시작하고나면 다른영웅으로 시작이됩니...


In [43]:
# 블리자드 사에서 올린 공지는 제외

df = df[df.user_id != "Blizzard Entertainment"]

In [46]:
# 헤비유저 파악 혹은 글 중복되어 올린 사람 파악하기

user_list = df['user_id'].unique()

for uid in user_list[:10]:
    print(df[df['user_id']==uid])
    
"""
df_by_user = df[df['contents']=='1']
df_by_user['timestamp'].tolist()


# 전처리 되도록이면 판다스로 하기. 유저 아이디 중복을 찾은다음에. 시간 차이가 몇 안나는 것들은 빼기(몇 분 이상 차이나면 없애는 방향으로)
"""

      link_id user_id           timestamp            title  \
2  4567016695   souse  2018.09.08 16시 22분  ㅈ같은 매칭좀 어캐좀 해봐요   

                                            contents  
2  왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야하죠? 왜 심지어 못하는새키랑 ...  
         link_id user_id           timestamp          title  \
3     4567026707    Best  2018.09.08 15시 13분      아나 스킨관련..   
68    4566996719    Best  2018.09.05 03시 46분             건의   
2628  4565604050    Best  2017.12.28 14시 30분             빠대   
2661  4565614129    Best  2017.12.26 01시 18분       한조 너프건의안   
2971  4565435532    Best  2017.11.30 04시 36분  지원가 너프에 관하여..   
3087  4565398541    Best  2017.11.18 15시 03분      소냐 너프 건의안   
4238  4564767573    Best  2017.06.19 07시 04분             빠대   
4584  4564622970    Best  2017.06.10 05시 05분          빠대관련    
4719  4564631707    Best  2017.05.31 18시 57분       루시우 건의안    
4796  4564630406    Best  2017.05.26 18시 19분       피드백 (건의)   

                                               contents  
3     신 스킨 살무

In [54]:
# 글 완전 중복인 사람 파악하기

df['contents'].drop_duplicates(keep='first')

2       왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야하죠? 왜 심지어 못하는새키랑 ...
3       신 스킨 살무사 아나 수면총 색 고정말고.  스킨 색 대로 해주셧으면 좋겠습니다. ...
4       승률 50% 만든다고 조합 !@내놓고5인팟은 여전히 솔팟 사이에서 개판치고다니고너프...
5       대만인 새끼들좀 매칭 안되게 해라.말도 안통하는 머저리 새끼들하고 왜 해야하는건데빠...
6       신영웅 이제 그만좀만드시고요. 기존영웅 쓸수나있게 손좀보시죠^^ 뭐가 더 급한건지 ...
7       삭제가 답이지 죽광도 리메이크해도 망맵 하나무라도 리메이크해도 망맵블랙하트는 예전에...
8       HOTS는 이전부터 영웅리그에 대한 문제가 많았습니다.크게 정리하자면 4가지가 있습...
9       분명 픽창에서는 제가 하고싶은 영웅을 선택했는데 시작하고나면 다른영웅으로 시작이됩니...
10      1년 한걸로 아는데 그래서 재밌음?이것만큼 강제한타맵은 없을 것 같은데 적어도 빠대...
11      진짜 개빡친다 7연패하고 2승하고 다시 7연패가 말이되냐?옛날에 승률올려놓은거 때문...
12      영웅과 스킨, 탈것 까지는 제대로 표기되지만, 예를 들어 미확인 이모티콘이 하나 있...
13      영웅리그에서 적팀에 트위치 스트리머 '뱅붕'을 의식하고, 아군 팀에서 계속 트롤하는...
14                        화이트메인 힐량이 게임내 플레이 중 혐오감을 조장합니다.
15      ㄹㅇ 이따위로 디자인 할꺼면 걍 이겜 롤 오브 더 스톰으로 바꾸셈영웅 특색은 씨1발...
16      ㅈ까 씨.발 저렇게 종횡무진 전장 휩쓸고 다니는 걸 어떻게 쳐 잡아? 이러다가 나중...
17      힐러가 탱커 급으로 버티는게 말이되냐 힐러 포지션 다 뒤엔데 화메는 앞임 어메이징 ...
18      빛나래를 플레이하는데 상대방 메피스토가 나왔습니다. 메피스토 전투스타일이 위치변환기...
19      역대급으로 

In [58]:
# 위에서 널값 처리한것 제거

df = df[df['contents'].notnull()]

In [76]:
# 엉망이 된 인덱스 다시 정렬
df = df.reset_index()

ValueError: cannot insert level_0, already exists

In [71]:
df.tail()

,level_0,index,link_id,user_id,timestamp,title,contents
5534,5534,5695,4564440767,카라딘,2017.04.06 03시 28분,발리라보고 데자뷰 느끼는 사람 없습니까?,빠대같은 초보존에선 종결 급 양학을 하고 다니면서 카운터를 준비할 수 있는 등급전에...
5535,5535,5696,4564493860,공격,2017.04.22 03시 08분,지금 ptr 게임이 안되는거같은데,서버가 게임 생성에 실패하였습니다 뜨면서 게임이안되요.. 저만그런거같지않는데. 공지...
5536,5536,5697,4564503573,고만하고자라,2017.04.22 02시 49분,가기전에 의견만남김.,"히오스 오픈베타때부터 해온사람으로서 몇자적고간다.트롤,탈주자,겜말아먹는넘들 단속안하..."
5537,5537,5698,4564493427,Messi,2017.04.21 20시 58분,게임 미참여로 정지 먹었는데,날짜가 지나도 안풀리네요. 영구정지먹은건가요?
5538,5538,5699,4564503257,geschichte,2017.04.21 19시 25분,한사람 나가면 게임무효를 시켜야지,빨리 끝나라고 가만히 있자니 튕기고..계속하자니 어차피 질 게임 시간만 길어지고 이...


In [ ]:
# datetime 수정용

import datetime
ex_date = df['timestamp'][0]
datetime.datetime()

In [72]:
for i in range(3):
    hannanum.nouns(df["contents"][i])

In [78]:
tokens = []

for i in list(df["contents"]):
    k = hannanum.nouns(i)
    tokens.append(k)

In [79]:
type(tokens)

list

In [80]:
tokens_df = pd.DataFrame({"tokens":tokens})

In [81]:
tokens_df.head()

,tokens
0,"[나, ㅈ같이못하, 새키들때문, 게임, 못하는새키랑, 같은판되서, 강등전, 매칭시스..."
1,"[신, 스킨, 살무사, 수면총, 색, 고정, 스킨, 색, 대, 해주셧으, 처음, 때..."
2,"[승률, 50, 조합, @내놓고5인팟, 솔팟, 사이, 개판치고다니고너프할, 애, 이..."
3,"[대만인, 새끼들좀, 매칭, 해, 말, 안통하, 머저리, 새끼들, 해야하는건데빠대,..."
4,"[영웅, 그만좀만드시고요, 기존영웅, 쓸수나있게, 손좀보시죠^^, 뭐, 것, 너프쳐..."


In [82]:
df["tokens_contents"] = tokens

In [83]:
id2word = corpora.Dictionary(df["tokens_contents"])

In [84]:
texts = df["tokens_contents"]

In [85]:
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]]


In [86]:
id2word[0]

'4년'

In [87]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('4년', 2),
  ('식', 1),
  ('못하는새키랑', 1),
  ('새키들때문', 1),
  ('게임', 2),
  ('요즘', 1),
  ('강등전', 1),
  ('같은판되서', 1),
  ('ㅈ같이못하', 1),
  ('밸런스', 1),
  ('새키들', 1),
  ('매칭', 1),
  ('나', 3),
  ('인성파탄나', 1),
  ('심한욕까', 1),
  ('사랑', 1),
  ('ㅈ같', 1),
  ('다이긴', 1),
  ('블리자드', 1),
  ('시스템', 1),
  ('히오스', 1),
  ('개사랑햇는데', 1),
  ('매칭시스템', 1),
  ('애정', 1),
  ('죽여버리고싶은욕구', 1),
  ('지경', 1),
  ('올해', 1)]]

In [105]:
# Build LDA model

%time lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=100, \
                                                  update_every=1, \
                                                  chunksize=100,\
                                                  passes=10, \
                                                  alpha='auto', \
                                                  per_word_topics=True)

CPU times: user 3min 7s, sys: 11.4 s, total: 3min 18s
Wall time: 3min 28s


In [106]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"스킨" + 0.023*"오니겐지" + 0.009*"2주차" + 0.008*"초상화" + 0.007*"진심" + '
  '0.007*"경찰디바" + 0.007*"공지" + 0.006*"궁" + 0.006*"누더기" + 0.005*"첫째주"'),
 (1,
  '0.013*"서버" + 0.011*"완료" + 0.007*"기준" + 0.007*"점수" + 0.006*"그거" + 0.006*"뭔" '
  '+ 0.006*"승리" + 0.006*"레벨" + 0.006*"일반" + 0.005*"바리"'),
 (2,
  '0.050*"히오스" + 0.044*"게임" + 0.036*"이벤트" + 0.034*"진짜" + 0.020*"나" + '
  '0.014*"유저들" + 0.013*"블리자드" + 0.012*"이" + 0.011*"뭐" + 0.008*"판"'),
 (3,
  '0.036*"매칭" + 0.024*"빠대" + 0.019*"시공" + 0.014*"조합" + 0.012*"일반전" + '
  '0.012*"영리" + 0.011*"시스템" + 0.011*"빠른대전" + 0.009*"파티" + 0.009*"등급전"'),
 (4,
  '0.062*"것" + 0.035*"수" + 0.019*"생각" + 0.016*"때" + 0.012*"오버워치" + 0.011*"퀘스트" '
  '+ 0.011*"데" + 0.010*"영웅" + 0.009*"이" + 0.008*"말"'),
 (5,
  '0.019*"인공지능" + 0.010*"궁금" + 0.007*"니" + 0.007*"탈주자" + 0.006*"니들" + '
  '0.006*"아이템" + 0.006*"발리라" + 0.005*"처리" + 0.005*"ㅋㅋㅋ" + 0.004*"로딩"'),
 (6,
  '0.025*"욕" + 0.008*"15판" + 0.007*"ㅠㅠ" + 0.005*"브락시스" + 0.004*"항전" + '
  '0.004*"용검" + 0.004*"장난" + 0.004*"정예" + 0.0

### coherence

In [107]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df["tokens_contents"], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.774218316624724

Coherence Score:  0.42473351325729614


In [108]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      28.575423        1       1 -0.312839  0.172675
8      14.826544        1       2 -0.152165 -0.166288
2      13.553361        1       3 -0.085774 -0.162972
3       9.222161        1       4  0.042231 -0.037156
1       7.769659        1       5  0.086458  0.020421
9       6.877936        1       6  0.078638  0.053823
7       6.374765        1       7  0.087104  0.053927
5       4.864499        1       8  0.084778  0.018982
0       4.076775        1       9  0.086481  0.027266
6       3.858877        1      10  0.085087  0.019323, topic_info=      Category         Freq   Term        Total  loglift  logprob
term                                                             
57079  Default  3993.000000      것  3993.000000  30.0000  30.0000
38693  Default  2009.000000    히오스  2009.000000  29.0000  29.0000
4839   Default  2763.000000     게임  2763.000000  28.0000  28.0000
23141  Default  2357.000000      수  2357.000000  27.0000  27.0000
13356  Default  1076.000000    이벤트  1076.000000  26.0000  26.0000
35126  Default  1007.000000     진짜  1007.000000  25.0000  25.0000
4560   Default  1171.000000      저  1171.000000  24.0000  24.0000
32443  Default   729.000000     매칭   729.000000  23.0000  23.0000
54550  Default  1178.000000     생각  1178.000000  22.0000  22.0000
8890   Default   978.000000      때   978.000000  21.0000  21.0000
46041  Default   606.000000      나   606.000000  20.0000  20.0000
18861  Default   494.000000     빠대   494.000000  19.0000  19.0000
23062  Default   361.000000     스킨   361.000000  18.0000  18.0000
56993  Default   555.000000     보상   555.000000  17.0000  17.0000
51528  Default   871.000000     유저   871.000000  16.0000  16.0000
42297  Default   355.000000     겐지   355.000000  15.0000  15.0000
41102  Default   388.000000     시공   388.000000  14.0000  14.0000
51430  Default   736.000000   오버워치   736.000000  13.0000  13.0000
36867  Default   750.000000    유저들   750.000000  12.0000  12.0000
32228  Default   680.000000    퀘스트   680.000000  11.0000  11.0000
46231  Default   291.000000  ㅋㅋㅋㅋㅋ   291.000000  10.0000  10.0000
25190  Default   665.000000      데   665.000000   9.0000   9.0000
17488  Default   600.000000     영웅   600.000000   8.0000   8.0000
16798  Default   517.000000   블리자드   517.000000   7.0000   7.0000
28638  Default   980.000000      이   980.000000   6.0000   6.0000
18431  Default   544.000000      뭐   544.000000   5.0000   5.0000
14851  Default   255.000000     디바   255.000000   4.0000   4.0000
32953  Default   215.000000      욕   215.000000   3.0000   3.0000
38169  Default   284.000000     조합   284.000000   2.0000   2.0000
42579  Default   206.000000   오니겐지   206.000000   1.0000   1.0000
...        ...          ...    ...          ...      ...      ...
17621  Topic10    71.363768    15판    72.170299   3.2436  -4.7730
21126  Topic10    61.802234     ㅠㅠ    62.608447   3.2418  -4.9169
32084  Topic10    39.057782   브락시스    39.864072   3.2344  -5.3758
15062  Topic10    36.323303     항전    37.129618   3.2328  -5.4484
38358  Topic10    33.582684     용검    34.389148   3.2311  -5.5268
18774  Topic10    31.618055     장난    32.424317   3.2296  -5.5871
3301   Topic10    29.837783     정예    30.644533   3.2281  -5.6450
50127  Topic10    26.185177     날조    27.002856   3.2240  -5.7756
42467  Topic10    25.404852   트롤러들    26.211203   3.2235  -5.8059
17015  Topic10    25.261688    단축키    26.067915   3.2234  -5.8115
50688  Topic10    24.249666   레스토랑    25.056413   3.2221  -5.8524
54915  Topic10    23.992696   전설스킨    24.799014   3.2217  -5.8631
43646  Topic10    22.083511      입    22.890430   3.2189  -5.9460
21311  Topic10    19.866416    접근성    20.686367   3.2143  -6.0518
44411  Topic10    19.293449     5초    20.100619   3.2138  -6.0811
39996  Topic10    19.250285     목록    20.056604   3.2138  -6.0833
3533   Topic10    18.253323  ㅠㅠㅠㅠㅠ    19.059468   3.2116  -6.1365
24687  To

In [ ]:
r'[ㄱ-ㅎㅏ-ㅣ]+'
#한글 정규표현식 찾아보기